# Desafio Properati

## 1) Importación de librerías

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import statsmodels.formula.api as smf
import pysal as ps
import re
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

E:\Miniconda3\envs\ds37\lib\site-packages\pysal\model\spvcm\abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


#### Importamos los datos y generamos el dataframe

In [2]:
df = pd.read_csv("properatti.csv")
df

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121215,121215,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,NaN,NaN,...,93.0,7699.115044,9354.838710,NaN,NaN,10000.0,http://www.properati.com.ar/1cja2_venta_depart...,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,Torre Forum Alcorta- Impecable 3 ambientes,https://thumbs4.properati.com/1/bjms0KnaAnlNoQ...
121216,121216,sell,house,Beccar,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3436080.0,NaN,NaN,...,360.0,1383.333333,1383.333333,NaN,NaN,NaN,http://www.properati.com.ar/1cja6_venta_casa_b...,Excelente e impecable casa en Venta en Las Lom...,Ruca Inmuebles | Venta | Lomas de San Isidro |...,https://thumbs4.properati.com/2/PCc3WuQDjpNZc4...
121217,121217,sell,apartment,Villa Urquiza,|Argentina|Capital Federal|Villa Urquiza|,Argentina,Capital Federal,3433775.0,"-34.5706388726,-58.4755963355",-34.570639,...,39.0,2858.695652,3371.794872,NaN,NaN,NaN,http://www.properati.com.ar/1cja7_venta_depart...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,https://thumbs4.properati.com/9/YAe_-2gRVykADP...
121218,121218,sell,apartment,Plaza Colón,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,NaN,NaN,NaN,...,48.0,1997.916667,1997.916667,NaN,NaN,NaN,http://www.properati.com.ar/1cja8_venta_depart...,"2 Amb al contrafrente, luminoso. El departame...",2 amb. C/ dep. de servicio al contrafrente| Re...,https://thumbs4.properati.com/8/Q12PTvU6BQJ0ib...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 26 columns):
Unnamed: 0                    121220 non-null int64
operation                     121220 non-null object
property_type                 121220 non-null object
place_name                    121197 non-null object
place_with_parent_names       121220 non-null object
country_name                  121220 non-null object
state_name                    121220 non-null object
geonames_id                   102503 non-null float64
lat-lon                       69670 non-null object
lat                           69670 non-null float64
lon                           69670 non-null float64
price                         100810 non-null float64
currency                      100809 non-null object
price_aprox_local_currency    100810 non-null float64
price_aprox_usd               100810 non-null float64
surface_total_in_m2           81892 non-null float64
surface_covered_in_m2         101313 no

In [183]:
df.loc[(df.price.isnull()) & ~(df.surface_total_in_m2.isnull())].title.str.extract(r'\$')

,operation,property_type,place_name,country_name,state_name,geonames_id,price,currency,price_aprox_local_currency,price_aprox_usd,...,localidad,barrio,lat,lon,amb_tit,dor_tit,amb_desc,dor_desc,alq,pozo
5,sell,house,Gualeguaychú,Argentina,Entre Ríos,3433657.0,NaN,NaN,NaN,NaN,...,Gualeguaychú,NaN,-33.014071,-58.519828,NaN,NaN,NaN,NaN,NaN,NaN
20,sell,house,Villa Giardino,Argentina,Córdoba,3832729.0,NaN,NaN,NaN,NaN,...,Villa Giardino,NaN,-31.052985,-64.499128,NaN,NaN,NaN,NaN,NaN,NaN
41,sell,apartment,Villa Carlos Paz,Argentina,Córdoba,3832791.0,NaN,NaN,NaN,NaN,...,Villa Carlos Paz,NaN,-31.400999,-64.478026,NaN,NaN,NaN,3.0,alq,NaN
62,sell,house,Berazategui,Argentina,Bs.As. G.B.A. Zona Sur,3436042.0,NaN,NaN,NaN,NaN,...,Berazategui,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,sell,house,Ituzaingó,Argentina,Bs.As. G.B.A. Zona Oeste,3433359.0,NaN,NaN,NaN,NaN,...,Ituzaingó,NaN,-34.642230,-58.703130,NaN,NaN,NaN,2.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121129,sell,apartment,Córdoba,Argentina,Córdoba,3860255.0,NaN,NaN,NaN,NaN,...,,None,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
121211,sell,apartment,Acassuso,Argentina,Bs.As. G.B.A. Zona Norte,3436508.0,NaN,NaN,NaN,NaN,...,San Isidro,Acassuso,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
121212,sell,house,Acassuso,Argentina,Bs.As. G.B.A. Zona Norte,3436508.0,NaN,NaN,NaN,NaN,...,San Isidro,Acassuso,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
121213,sell,house,Martínez,Argentina,Bs.As. G.B.A. Zona Norte,3430813.0,NaN,NaN,NaN,NaN,...,San Isidro,Martínez,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN


#### Obtenemos los porcentajes de datos faltantes

In [4]:
for cols in df.columns:
    nulos = df[cols].isnull().sum()
    porcentaje = nulos/len(df)
    print(cols, porcentaje*100)

Unnamed: 0 0.0
operation 0.0
property_type 0.0
place_name 0.018973766705164163
place_with_parent_names 0.0
country_name 0.0
state_name 0.0
geonames_id 15.440521366111204
lat-lon 42.52598581092229
lat 42.52598581092229
lon 42.52598581092229
price 16.837155584886982
currency 16.83798053126547
price_aprox_local_currency 16.837155584886982
price_aprox_usd 16.837155584886982
surface_total_in_m2 32.44349117307375
surface_covered_in_m2 16.422207556508827
price_usd_per_m2 43.394654347467416
price_per_m2 27.686850354726943
floor 93.48374855634384
rooms 60.90579112357697
expenses 88.23461475004125
properati_url 0.0
description 0.001649892756970797
title 0.0
image_thumbnail 2.56723312984656


#### Desagregamos los datos contenidos en `place_with_parent_names`

In [5]:
place_split = df.place_with_parent_names.str.split('|', expand=True).rename({1:'pais', 2:'provincia',
                                                               3:'localidad', 4:'barrio'}, axis=1).drop([0,5,6], axis=1)

place_split.loc[(place_split.barrio == ''), 'barrio'] = np.nan

#### Los adjuntamos al df original

In [6]:
df = df.join(place_split)
df

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,rooms,expenses,properati_url,description,title,image_thumbnail,pais,provincia,localidad,barrio
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,Argentina,Capital Federal,Mataderos,NaN
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,NaN
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...,Argentina,Capital Federal,Mataderos,NaN
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...,Argentina,Capital Federal,Liniers,NaN
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...,Argentina,Buenos Aires Costa Atlántica,Mar del Plata,Centro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121215,121215,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,NaN,NaN,...,NaN,10000.0,http://www.properati.com.ar/1cja2_venta_depart...,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,Torre Forum Alcorta- Impecable 3 ambientes,https://thumbs4.properati.com/1/bjms0KnaAnlNoQ...,Argentina,Capital Federal,Belgrano,NaN
121216,121216,sell,house,Beccar,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3436080.0,NaN,NaN,...,NaN,NaN,http://www.properati.com.ar/1cja6_venta_casa_b...,Excelente e impecable casa en Venta en Las Lom...,Ruca Inmuebles | Venta | Lomas de San Isidro |...,https://thumbs4.properati.com/2/PCc3WuQDjpNZc4...,Argentina,Bs.As. G.B.A. Zona Norte,San Isidro,Beccar
121217,121217,sell,apartment,Villa Urquiza,|Argentina|Capital Federal|Villa Urquiza|,Argentina,Capital Federal,3433775.0,"-34.5706388726,-58.4755963355",-34.570639,...,NaN,NaN,http://www.properati.com.ar/1cja7_venta_depart...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,https://thumbs4.properati.com/9/YAe_-2gRVykADP...,Argentina,Capital Federal,Villa Urquiza,NaN
121218,121218,sell,apartment,Plaza Colón,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,NaN,NaN,NaN,...,NaN,NaN,http://www.properati.com.ar/1cja8_venta_depart...,"2 Amb al contrafrente, luminoso. El departame...",2 amb. C/ dep. de servicio al contrafrente| Re...,https://thumbs4.properati.com/8/Q12PTvU6BQJ0ib...,Argentina,Buenos Aires Costa Atlántica,Mar del Plata,Plaza Colón


Este código hace exactamente lo mismo y sería la forma más limpia de hacerlo
```python
df.place_with_parent_names.str.split('|', expand=True).rename({1:'pais', 2:'provincia',
                                                               3:'partido', 4:'localidad',
                                                               5:'barrio', 6:'zona'}, axis=1).drop(0, axis=1)
#Alternativa
place_split = df.place_with_parent_names.apply(lambda x: x.split('|')).apply(pd.Series)\
                            .rename({1:'pais', 2:'provincia', 3:'partido', 4:'localidad',5:'barrio', 6:'zona'}, axis=1)\
                            .drop(0, axis=1)
```

#### Eliminamos las columnas ahora innecesarias

In [7]:
df.drop(["place_with_parent_names", "Unnamed: 0"], axis=1, inplace=True)

#### Separamos las coordenadas albergadas en la variable `lat-lon` que aparentemente tienen mayor precisión que las que se ofrecen en las variables `lat` y `lon`

Posteriormente, eliminamos las variables redundantes del df original y adjuntamos las nuevas en formato `float`

In [8]:
lat_lon = df['lat-lon'].str.split(',', expand=True)
lat_lon = lat_lon.rename({0:'lat', 1:'lon'}, axis=1)
df = df.drop(['lat-lon', 'lat', 'lon'], axis=1).join(lat_lon)

In [9]:
df['lat'] = df.lat.astype(np.float)
df['lon'] = df.lon.astype(np.float)

#### Generamos expresiones regulares para extraer datos de las descripciones y títulos.
Empezamos por los patrones de referentes a ambientes y dormitorios con una expresión bastante sencilla

In [10]:
pat_ambientes = r'\b(\d{1,2})\s*amb'
pat_dormitorios = r'\b(\d{1,2})\s*dor'

In [8]:
#pattern_usd = r'[u$s dolares]?((?!\()\d{1,3}[,.]\d{1,3}[,.]?\d{3}?(?!\)))|((?!\()\d{1,3}[,.]\d{3}(?!\)))|((?!\()\d{5,}(?!\)))'
#pat_tag_usd = r'(usd|u\$s|us\$|dolares|dólares|us)'
#pat_tag_ars = r''

#### Extraemos los datos a las nuevas columnas `amb_tit`, `dor_tit`, `amb_desc` y `dor_desc`

In [11]:
df['amb_tit'] = df.title.str.extract(pat_ambientes, re.IGNORECASE, expand=True).astype(np.float)
df['dor_tit'] = df.title.str.extract(pat_dormitorios, re.IGNORECASE, expand=True).astype(np.float)

df['amb_desc'] = df.description.str.extract(pat_ambientes, re.IGNORECASE, expand=True).astype(np.float)
df['dor_desc'] = df.description.str.extract(pat_dormitorios, re.IGNORECASE, expand=True).astype(np.float)

Verificamos los valores extraídos

In [12]:
df.loc[~(df.amb_tit.isnull())].filter(['amb_tit']).sort_values('amb_tit').amb_tit.unique()

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 14.,
       15.])

In [13]:
df.loc[~(df.dor_tit.isnull())].filter(['dor_tit']).sort_values('dor_tit').dor_tit.unique()

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.])

In [14]:
df.loc[~(df.amb_desc.isnull())].filter(['amb_desc']).sort_values('amb_desc').amb_desc.unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       14., 16., 17., 18., 20., 22., 23., 24., 25., 28., 32., 36., 37.,
       50., 53., 65., 70., 75., 84., 90.])

In [15]:
df.loc[~(df.dor_desc.isnull())].filter(['dor_desc']).sort_values('dor_desc').dor_desc.unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 13.,
       14., 15., 18., 20., 22., 24., 25., 27., 30., 31., 32., 33., 34.,
       35., 40., 41., 42., 43., 44., 45., 48., 50., 51., 52., 53., 54.,
       57., 60., 62., 65., 70., 72., 74., 75., 76., 77., 78., 80., 83.,
       84., 85., 87., 90., 93., 95.])

Revisamos cuántos valores conseguimos de las distintas fuentes

In [88]:
df.loc[~(df.amb_tit.isnull())].title.count(), df.loc[~(df.dor_tit.isnull())].title.count()

(21799, 7764)

In [89]:
df.loc[~(df.amb_desc.isnull())].title.count(), df.loc[~(df.dor_desc.isnull())].title.count()

(37438, 40448)

Como podemos observar, los valores extraidos de la descripcion toman valores mucho mayores a los esperados. Debemos revisar qué es lo que está obteniendo, que a priori, parecería ser la cantidad de metros cuadrados de dichos ambientes

In [166]:
i = np.random.randint(0, 21799)
df[~(df.amb_desc.isnull())].iloc[i].description, df[~(df.amb_desc.isnull())].iloc[i].amb_desc

('tor930- Apart unico de 38m2 Financiado en pesosAnticipo financiado en 10 cuotitasy  CUOTITAS DESDE $ 10.450- 1 AMBIENTE DIVISIBLE DE 38 M2 CON BALCON Y SIN BALCONTODO EXTERNOY 2 AMBIENTESSOBRE AVENIDA ? PXMO CENTROEMPRESA CONSTRUCTORA VENDE EN FORMADIRECTA, SIN INTERMEDIARIOS1 Y 2 AMBIENTES FINANCIADOS EN VARIOS AOSULTIMAS UNIDADES PRE-VENTA  DEPTO TODO FINANCIADO PARA INVERTIRESTA ES LA FORMA DE TENER TU DEPANUEVO PROYECTO SIN  REQUISITOS EN  CUOTAS DESDE $10.450.-Anticipate y Reserva hoy tu unidadCon exclusivos beneficiosEXCLUSIVA PRE VENTA DE POZOLaundry ? Sum ? SolariumSauna ? Parrilla ? JacuzziTENEMOS OTROS EDIFICIOSConsultanos:Te:    15     15 Mail: ;Pag:   www.http:// .Publicado por Emprendimientos Financiados a traves INMOMAP',
 1.0)

In [163]:
df['alq'] = df.description.str.extract(r'(\balq)', re.IGNORECASE, expand=True)

In [47]:
df['pozo'] = df.description.str.extract(r'(\ben\spozo)', re.IGNORECASE, expand=True)

In [116]:
df.drop(df.loc[~(df.pozo.isnull())].index, axis=0, inplace=True)

In [171]:
df.loc[(df.rooms.isnull()) & (~df.amb_tit.isnull())]

,operation,property_type,place_name,country_name,state_name,geonames_id,price,currency,price_aprox_local_currency,price_aprox_usd,...,localidad,barrio,lat,lon,amb_tit,dor_tit,amb_desc,dor_desc,alq,pozo
0,sell,PH,Mataderos,Argentina,Capital Federal,3430787.0,62000.0,USD,1093959.00,62000.0,...,Mataderos,NaN,-34.661824,-58.508839,2.0,NaN,2.0,NaN,NaN,NaN
2,sell,apartment,Mataderos,Argentina,Capital Federal,3430787.0,72000.0,USD,1270404.00,72000.0,...,Mataderos,NaN,-34.652262,-58.522982,2.0,NaN,2.0,NaN,NaN,NaN
3,sell,PH,Liniers,Argentina,Capital Federal,3431333.0,95000.0,USD,1676227.50,95000.0,...,Liniers,NaN,-34.647797,-58.516424,3.0,NaN,3.0,NaN,NaN,NaN
4,sell,apartment,Centro,Argentina,Buenos Aires Costa Atlántica,3435548.0,64000.0,USD,1129248.00,64000.0,...,Mar del Plata,Centro,-38.002626,-57.549447,2.0,NaN,NaN,1.0,NaN,NaN
12,sell,apartment,Martínez,Argentina,Bs.As. G.B.A. Zona Norte,3430813.0,115000.0,USD,2029117.50,115000.0,...,San Isidro,Martínez,-34.486019,-58.503814,2.0,NaN,2.0,30.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121131,sell,apartment,Recoleta,Argentina,Capital Federal,3429595.0,325000.0,USD,5734462.50,325000.0,...,Recoleta,NaN,-34.590628,-58.397539,3.0,NaN,4.0,2.0,NaN,NaN
121138,sell,apartment,Saavedra,Argentina,Capital Federal,3429422.0,208000.0,USD,3670056.00,208000.0,...,Saavedra,NaN,NaN,NaN,2.0,NaN,2.0,NaN,NaN,NaN
121153,sell,apartment,Barrio Norte,Argentina,Capital Federal,3436109.0,165000.0,USD,2911342.50,165000.0,...,Barrio Norte,NaN,-34.599214,-58.402519,3.0,NaN,NaN,NaN,NaN,NaN
121215,sell,apartment,Belgrano,Argentina,Capital Federal,3436077.0,870000.0,USD,15350715.00,870000.0,...,Belgrano,NaN,NaN,NaN,3.0,NaN,NaN,2.0,NaN,NaN


In [14]:
a = df.filter(['ambientes', 'dormitorios', 'rooms']).loc[~(df.rooms.isnull()) &\
                                                         ~(df.ambientes.isnull())].ambientes.astype(np.float64)\
    == df.filter(['ambientes', 'dormitorios', 'rooms']).loc[~(df.rooms.isnull()) & ~(df.ambientes.isnull())].rooms

In [15]:
a.sum()/len(a)

0.9498057734785589

El resultado anterior indica que rooms y ambientes es lo mismo, por lo que podemos rellenar los `NaN` de *'rooms'* con *'ambientes'*

In [16]:
df.loc[~(df.dormitorios.isnull())].filter(['description', 'dormitorios'])

,description,dormitorios
1,Venta de departamento en décimo piso al frente...,2
9,MEDNOZA AL 7600A UNA CUADRA DE CALLE MENDOZAWH...,2
34,"Amplio PH de 5 dormitorios, ubicado a 3 cuadra...",5
73,SE VENDE CASA EN EL MICROCENTRO DE 150 mts2 CO...,2
83,IDEAL INVERSORES o ESTUDIANTES APTA CREDITO An...,2
...,...,...
120331,Gran departamento en venta de 3 dormitoriosDep...,3
120962,Hermosa Casa sobre gran lote en esquina muy lu...,4
120966,HDH Propiedades presenta a la venta casa apta...,3
121101,Excelente casa de dos plantas a entrenar en Sa...,3


In [17]:
#df.rooms.fillna(df.ambientes, inplace=True)
#df.filter(['rooms', 'ambientes', 'dormitorios']).loc[~(df.rooms.isnull())]

In [18]:
df['re_usd'] = df.title.str.extract(pat_tag_usd, re.IGNORECASE)

In [19]:
df.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail', 'ambientes', 'dormitorios', 're_usd'],
      dtype='object')

In [20]:
df.loc[~(df.re_usd.isnull())&(df.price.isnull())].filter(['re_usd']).values

array([['US'],
       ['US'],
       ['US'],
       ['US'],
       ['US'],
       ['us'],
       ['us'],
       ['us'],
       ['us'],
       ['us'],
       ['us'],
       ['us'],
       ['us'],
       ['us'],
       ['us'],
       ['us'],
       ['us'],
       ['us'],
       ['us'],
       ['us'],
       ['us'],
       ['us'],
       ['Us'],
       ['US'],
       ['us'],
       ['US'],
       ['us'],
       ['US'],
       ['US'],
       ['us'],
       ['us'],
       ['us'],
       ['us'],
       ['US'],
       ['US'],
       ['US'],
       ['us'],
       ['US'],
       ['US'],
       ['US'],
       ['us'],
       ['US'],
       ['US'],
       ['us'],
       ['us'],
       ['US'],
       ['us'],
       ['US'],
       ['us'],
       ['US'],
       ['US'],
       ['US'],
       ['US'],
       ['us'],
       ['US'],
       ['US'],
       ['US'],
       ['US'],
       ['US'],
       ['us'],
       ['us'],
       ['US'],
       ['us'],
       ['US'],
       ['us'],
       ['US'],
       ['u

In [21]:
df.iloc[8]['title']

'JOSE HERNANDEZ   1400 DOS AMBIENTES ESTRENAR , CATEGORIA , VENTA'

In [22]:
df['amb_description'] = df.description.str.extract(pat_ambientes, re.IGNORECASE, expand=True)

In [23]:
#df.rooms.fillna(df.amb_description, inplace=True)

In [24]:
df.loc[~(df.rooms.isnull())].rooms.shape

(47390,)

In [25]:
pat_amb_esp = r'ambientes:([\s]?\d{1,2})'

In [26]:
df['amb_des_es'] = df.description.str.extract(pat_amb_esp, re.IGNORECASE, expand=True)

In [64]:
df.loc[~(df.amb_des_es.isnull())].sample(1).filter(['description']).values

AttributeError: 'DataFrame' object has no attribute 'amb_des_es'

In [28]:
b = df.loc[~(df.rooms.isnull()) &\
                                                         ~(df.amb_des_es.isnull())].amb_des_es.astype(np.float64)\
    == df.loc[~(df.rooms.isnull()) & ~(df.amb_des_es.isnull())].rooms

In [30]:
b.sum()/b.count()

0.4626865671641791